In [1]:
import pandas as pd
import joblib
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_absolute_error

from Funcoes_Comuns import avaliar_modelo, registrar_modelo

#### 1. Recuperar base já pré-processada

In [2]:
# Obter dados
df_enem = pd.read_pickle('Bases\\Finais\\enem_microdados_2023.pkl')

#### 2. Label Encoding
- Apenas transforma variáveis categóricas em numéricas, não há significado numérico
- Processamento necessário para modelos aplicados: árvore de decisão

In [3]:
# Aplicar o LabelEncoder para converter as colunas categóricas em numéricas
# Salvar os encoders para possível uso posterior
label_encoders = {}
categorical_columns = df_enem.select_dtypes(include=['category']).columns

for col in categorical_columns:
    le = LabelEncoder()
    df_enem[col] = le.fit_transform(df_enem[col])
    label_encoders[col] = le  # Salvar o encoder para a coluna

In [4]:
# Verificar valores faltantes após Label Encoding
print("Verificação de valores faltantes:")
print(f"Total de valores nulos: {df_enem.isnull().sum().sum()}")

Verificação de valores faltantes:
Total de valores nulos: 0


In [5]:
df_enem.head()

,NUM_NOTA_CH,NUM_NOTA_CN,NUM_NOTA_LC,NUM_NOTA_MT,NUM_NOTA_REDACAO,BIN_Q001_DUMMY_H,BIN_Q002_DUMMY_H,BIN_Q018,BIN_Q020,BIN_Q021,...,NUM_Q011,NUM_Q012,NUM_Q013,NUM_Q014,NUM_Q015,NUM_Q016,NUM_Q017,NUM_Q019,NUM_Q022,NUM_Q024
0,508.5,459.0,507.2,466.7,880.0,False,False,False,False,False,...,0,1,0,0,0,0,0,1,3,0
1,379.2,402.5,446.9,338.3,560.0,False,False,False,False,False,...,0,1,0,0,0,0,0,1,1,0
2,667.6,608.2,607.9,691.9,780.0,False,False,False,False,False,...,0,1,0,1,0,0,0,1,1,1
3,553.1,515.7,544.4,437.0,880.0,False,False,False,False,False,...,0,1,0,0,0,0,0,1,3,0
4,576.3,523.8,596.5,628.1,600.0,False,False,False,False,False,...,0,1,0,1,0,0,0,1,1,0


In [6]:
variaveis_alvo = ['NUM_NOTA_MT', 'NUM_NOTA_LC', 'NUM_NOTA_CN', 'NUM_NOTA_CH', 'NUM_NOTA_REDACAO']

# separar em treino e teste
X = df_enem.drop(columns=variaveis_alvo)
y = df_enem[variaveis_alvo]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajuste de tipo para MLflow
# Converter colunas inteiras para float
X_train = X_train.astype({col: 'float' for col in X_train.select_dtypes('int').columns})
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

#### 3. Modelo de Árvore simples como base

In [7]:
# Treinar modelos de árvore de decisão para regressão
modelo_arvore_decisao = DecisionTreeRegressor(random_state=42,
                                               max_depth=5, 
                                               min_samples_split=10, 
                                               min_samples_leaf=5,
                                               splitter='best')

start_time = time.time()

modelo_arvore_decisao.fit(X_train, y_train['NUM_NOTA_CH'])

tempo_treino = time.time() - start_time

In [8]:
# Avaliação do modelo
y_pred_arvore_decisao = modelo_arvore_decisao.predict(X_test)

In [9]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento,
                    parametros={**modelo_arvore_decisao.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                    X_train=X_train,
                    y_train=y_train,
                    y_test=y_test,
                    y_pred=y_pred_arvore_decisao,
                    variavel_alvo='NUM_NOTA_CH', 
                    modelo=modelo_arvore_decisao,
                    nome_modelo='modelo_arvore_decisao_base',
                    descricao_modelo='Decision Tree Regressor')

Erro ao registrar o modelo no MLflow: API request to http://127.0.0.1:9080/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='127.0.0.1', port=9080): Max retries exceeded with url: /api/2.0/mlflow/experiments/get-by-name?experiment_name=Notas+CH+ENEM+2023 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002B68D56C250>: Failed to establish a new connection: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente'))
Rastreamento do MLflow finalizado.


In [10]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_arvore_decisao.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred_arvore_decisao, "teste")

MAE (treino): 57.7430
RMSE (treino): 72.8224
R2 (treino): 0.2601
MAE (teste): 57.6438
RMSE (teste): 72.6746
R2 (teste): 0.2591


In [11]:
# Salvar o modelo otimizado como um arquivo pickle
joblib.dump(modelo_arvore_decisao, 'Modelos\\modelo_arvore_decisao_base.pkl')

['Modelos\\modelo_arvore_decisao_base.pkl']

#### Busca melhor ccp_alpha

##### ccp_alpha:

- Controla a poda da árvore, removendo nós com impacto mínimo no erro.
- Valores maiores resultam em árvores mais simples e generalizáveis.
- O método cost_complexity_pruning_path é usado para calcular os valores possíveis de ccp_alpha.

In [12]:
path = modelo_arvore_decisao.cost_complexity_pruning_path(X_train, y_train['NUM_NOTA_CH'])  # CCP Path na base de treino
ccp_alphas, impurities = path.ccp_alphas, path.impurities

In [13]:
ccp_alphas = pd.Series(ccp_alphas).unique()
ccp_alphas

array([0.00000000e+00, 7.86228603e-02, 1.67620614e-01, 3.61391307e-01,
       6.81614093e-01, 9.75407282e-01, 1.14725498e+00, 1.27219204e+00,
       1.55189437e+00, 1.77448604e+00, 2.68224813e+00, 2.94715202e+00,
       4.37682104e+00, 4.47340671e+00, 5.17929702e+00, 5.31924231e+00,
       6.04964765e+00, 7.51181432e+00, 1.04898993e+01, 1.56790651e+01,
       1.69540100e+01, 2.01405984e+01, 2.50485001e+01, 2.72318991e+01,
       3.73134523e+01, 4.55326390e+01, 6.31282810e+01, 8.98753766e+01,
       9.38742721e+01, 9.42068882e+01, 2.89662230e+02, 9.88193098e+02])

In [14]:
# order the values of ccp_alphas
ccp_alphas = sorted(ccp_alphas, reverse=False)
ccp_alphas

[np.float64(0.0),
 np.float64(0.07862286029428978),
 np.float64(0.16762061358154057),
 np.float64(0.36139130731795177),
 np.float64(0.6816140931796966),
 np.float64(0.9754072815338191),
 np.float64(1.1472549784255648),
 np.float64(1.2721920404640485),
 np.float64(1.5518943696056553),
 np.float64(1.7744860430268545),
 np.float64(2.682248126403337),
 np.float64(2.9471520233014843),
 np.float64(4.376821037894729),
 np.float64(4.473406714650537),
 np.float64(5.179297015841996),
 np.float64(5.319242309856861),
 np.float64(6.049647649577594),
 np.float64(7.511814320948247),
 np.float64(10.489899343926254),
 np.float64(15.679065144324113),
 np.float64(16.954010029821575),
 np.float64(20.14059841320045),
 np.float64(25.048500144516765),
 np.float64(27.231899051805385),
 np.float64(37.31345226609392),
 np.float64(45.53263902688332),
 np.float64(63.12828095102441),
 np.float64(89.87537657716507),
 np.float64(93.87427210788564),
 np.float64(94.20688820779878),
 np.float64(289.6622301897323),
 np.

In [15]:
len(ccp_alphas)  # Número de valores únicos de alpha

32

In [16]:
obter_melhor_alpha = False

In [17]:
if obter_melhor_alpha:
    metricas_calculadas = []
    contador = 0

    for ccp_alpha in ccp_alphas:
        print(f"Iteração {contador} de {len(ccp_alphas)}")
        contador += 1

        # Treinar o modelo de árvore de decisão com o valor de ccp_alpha atual
        modelo_arvore_decisao = DecisionTreeRegressor(random_state=42, ccp_alpha=ccp_alpha)
        modelo_arvore_decisao.fit(X_train, y_train['NUM_NOTA_CH'])
        y_pred_arvore_decisao = modelo_arvore_decisao.predict(X_test)

        # Avaliação do modelo
        mae_arvore_decisao_teste = mean_absolute_error(y_test['NUM_NOTA_CH'], y_pred_arvore_decisao)
        print(f"MAE (teste): {mae_arvore_decisao_teste:.2f}")
        # rmse_arvore_decisao_teste = root_mean_squared_error(y_test['NUM_NOTA_CH'], y_pred_arvore_decisao)
        # r2_arvore_decisao_teste = r2_score(y_test['NUM_NOTA_CH'], y_pred_arvore_decisao)

        # Salvar métricas
        metricas_calculadas.append({
            'ccp_alpha': ccp_alpha,
            'MAE': mae_arvore_decisao_teste,
            # 'RMSE': rmse_arvore_decisao_teste,
            # 'R2': r2_arvore_decisao_teste
        })

        # MAE atual maior que os dois anteriores parar processamento
        if contador > 2 and metricas_calculadas[-1]['MAE'] > metricas_calculadas[-2]['MAE'] and metricas_calculadas[-2]['MAE'] > metricas_calculadas[-3]['MAE']:
            print(f"Parando o loop no alpha {ccp_alpha} com MAE {mae_arvore_decisao_teste:.2f}")
            break

In [18]:
if obter_melhor_alpha:
    # Melhor valor de alpha para MAE mínimo
    melhor_mae = min(metricas_calculadas, key=lambda x: x['MAE'])

    # Melhor valor de alpha para RMSE mínimo
    # melhor_rmse = min(metricas_calculadas, key=lambda x: x['RMSE'])

    # Melhor valor de alpha para R2 máximo
    # melhor_r2 = max(metricas_calculadas, key=lambda x: x['R2'])

    # Melhores valores de alpha
    print(f"Melhor alpha: {melhor_mae['ccp_alpha']:.4f}, com MAE: {melhor_mae['MAE']:.2f}")
    # print(f"Melhor alpha: {melhor_rmse['ccp_alpha']:.4f}, com RMSE: {melhor_rmse['RMSE']:.2f}")
    # print(f"Melhor alpha: {melhor_r2['ccp_alpha']:.4f}, com R2: {melhor_r2['R2']:.2f}")

Melhor CCP Alpha para menor MAE: 0.3436

In [19]:
if not obter_melhor_alpha:
    ccp_alpha = 0.3436

In [20]:
# Treinar árvore de decisão com o melhor alpha
modelo_arvore_decisao_alpha = DecisionTreeRegressor(random_state=42,
                                                    max_depth=5,
                                                    min_samples_split=10, 
                                                    min_samples_leaf=5,
                                                    splitter='best',
                                                    # ccp_alpha=melhor_mae['ccp_alpha'])
                                                    ccp_alpha=ccp_alpha)

start_time = time.time()

modelo_arvore_decisao_alpha.fit(X_train, y_train['NUM_NOTA_CH'])

tempo_treino = time.time() - start_time

In [21]:
# Avaliação do modelo
y_pred_arvore_decisao_alpha = modelo_arvore_decisao_alpha.predict(X_test)

In [22]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento,
                 parametros={**modelo_arvore_decisao_alpha.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                    X_train=X_train,
                    y_train=y_train,
                    y_test=y_test,
                    y_pred=y_pred_arvore_decisao_alpha,
                    variavel_alvo='NUM_NOTA_CH',
                    modelo=modelo_arvore_decisao_alpha,
                    nome_modelo='modelo_arvore_decisao_alpha',
                    descricao_modelo='Decision Tree Regressor com alpha otimizado')

Erro ao registrar o modelo no MLflow: API request to http://127.0.0.1:9080/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='127.0.0.1', port=9080): Max retries exceeded with url: /api/2.0/mlflow/experiments/get-by-name?experiment_name=Notas+CH+ENEM+2023 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002B68D5268D0>: Failed to establish a new connection: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente'))
Rastreamento do MLflow finalizado.


In [23]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_arvore_decisao_alpha.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred_arvore_decisao_alpha, "teste")

MAE (treino): 57.7446
RMSE (treino): 72.8241
R2 (treino): 0.2600
MAE (teste): 57.6427
RMSE (teste): 72.6738
R2 (teste): 0.2592


In [24]:
# Salvar o modelo otimizado como um arquivo pickle
joblib.dump(modelo_arvore_decisao_alpha, 'Modelos\\modelo_arvore_decisao_alpha.pkl')

['Modelos\\modelo_arvore_decisao_alpha.pkl']

Grid Search

In [25]:
# Configurar os parâmetros para o GridSearchCV

param_grid = {
    'ccp_alpha': [0.2, 0.3436, 0.4],
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'splitter': ['best', 'random']
}

In [26]:
# Validação cruzada para regressão
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Criar o modelo de árvore de decisão
modelo_arvore_decisao_grid = DecisionTreeRegressor(random_state=42)

In [27]:
# Realizar o GridSearchCV
grid_search = GridSearchCV(estimator=modelo_arvore_decisao_grid,
                           param_grid=param_grid,
                           scoring='r2',
                           cv=kf,
                           return_train_score=True,
                           n_jobs=-1,
                           verbose=1)

In [28]:
# Executar o GridSearchCV
grid_search.fit(X_train, y_train['NUM_NOTA_CH'])

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=DecisionTreeRegressor(random_state=42), n_jobs=-1,
             param_grid={'ccp_alpha': [0.2, 0.3436, 0.4],
                         'max_depth': [3, 5, 7, 10],
                         'min_samples_leaf': [1, 2, 5],
                         'min_samples_split': [2, 5, 10],
                         'splitter': ['best', 'random']},
             return_train_score=True, scoring='r2', verbose=1)

In [29]:
# Melhores parâmetros encontrados
try:
    melhores_parametros = grid_search.best_params_
    print(f"Melhores parâmetros: {melhores_parametros}")
except:
    melhores_parametros = {'ccp_alpha': 0.2, 'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 2, 'splitter': 'best'}
    print(f"Erro ao obter melhores parâmetros, usando valores calculados anteriormente:\n {melhores_parametros}")

Melhores parâmetros: {'ccp_alpha': 0.2, 'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 2, 'splitter': 'best'}


In [30]:
# Treinar o modelo com os melhores parâmetros
modelo_arvore_decisao_best = DecisionTreeRegressor(random_state=42, **melhores_parametros)

start_time = time.time()

modelo_arvore_decisao_best.fit(X_train, y_train['NUM_NOTA_CH'])

tempo_treino = time.time() - start_time

In [31]:
# Avaliação do modelo final
y_pred_arvore_decisao_best = modelo_arvore_decisao_best.predict(X_test)

In [32]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento,
                     parametros={**modelo_arvore_decisao_best.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                     X_train=X_train,
                     y_train=y_train,
                     y_test=y_test,
                     y_pred=y_pred_arvore_decisao_best,
                     variavel_alvo='NUM_NOTA_CH',
                     modelo=modelo_arvore_decisao_best,
                     nome_modelo='modelo_arvore_decisao_grid',
                     descricao_modelo='Decision Tree Regressor com GridSearchCV')

Erro ao registrar o modelo no MLflow: API request to http://127.0.0.1:9080/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='127.0.0.1', port=9080): Max retries exceeded with url: /api/2.0/mlflow/experiments/get-by-name?experiment_name=Notas+CH+ENEM+2023 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002B6A779EC50>: Failed to establish a new connection: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente'))
Rastreamento do MLflow finalizado.


In [33]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_arvore_decisao_best.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred_arvore_decisao_best, "teste")

MAE (treino): 56.5474
RMSE (treino): 71.5282
R2 (treino): 0.2861
MAE (teste): 56.5914
RMSE (teste): 71.5510
R2 (teste): 0.2819


In [34]:
# Salvar o modelo otimizado como um arquivo pickle
joblib.dump(modelo_arvore_decisao_best, 'Modelos\\modelo_arvore_decisao_best.pkl')

['Modelos\\modelo_arvore_decisao_best.pkl']